In [1]:
import boto3
import json

client = boto3.client('rds')

#### Use AWS Secrets Manager to generate a complex password:

In [5]:
secret_manager = boto3.client('secretsmanager')
response = secret_manager.get_random_password(
    ExcludePunctuation=True,
    PasswordLength=41)
admin_password = response['RandomPassword']

#### Create a VPC

In [9]:
# create VPC
region_aws = 'us-east-1'

ec2 = boto3.resource('ec2', region_name=region_aws)
ec2_client = boto3.client('ec2', region_name=region_aws)

In [10]:
vpc = ec2.create_vpc(CidrBlock='174.17.0.0/16')
vpc.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookBookVPC"}])
vpc.wait_until_available()

In [11]:
# Crear routable
routetable_1 = vpc.create_route_table()
routetable_2 = vpc.create_route_table()
routetable_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-RT-1"}])
routetable_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbookVPC-RT-2"}])

[ec2.Tag(resource_id='rtb-05707f5d7cd6c12b9', key='Name', value='AWSCookbookVPC-RT-2')]

In [14]:
# Create two subnets
subnet_1 = ec2.create_subnet(
    CidrBlock='174.17.5.0/24', 
    VpcId=vpc.id,
    AvailabilityZone= region_aws + 'a'
)

subnet_2 = ec2.create_subnet(
    CidrBlock='174.17.6.0/24', 
    VpcId=vpc.id,
    AvailabilityZone= region_aws + 'b'
)
subnet_1.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-SN-Private-1"}])
subnet_2.create_tags(Tags=[{"Key": "Name", "Value": "AWSCookbook-SN-Private-2"}])

In [16]:
# Asociar las tablas de rutas con las subred:
routetable_1.associate_with_subnet(SubnetId = subnet_1.id)
routetable_2.associate_with_subnet(SubnetId = subnet_2.id)

ec2.RouteTableAssociation(id='rtbassoc-0b3f8212518fbe196')

#### Create a database subnet group

In [17]:
# Create a database subnet group specifying the VPC subnets to use for the cluster. 
# Database subnet groups simplify the placement of RDS elastic network interfaces (ENIs)

client_rds = boto3.client('rds')
subnet_group_name = 'database-subnet-group-01'

response = client_rds.create_db_subnet_group(
    DBSubnetGroupName=subnet_group_name,
    DBSubnetGroupDescription='Subnet group for database instance',
    SubnetIds=[
        subnet_1.id,
        subnet_2.id
    ]
)

In [20]:
# Create a VPC security group for the database
response = ec2_client.create_security_group(
    GroupName='database-security-group-01',
    Description='Security group for database instance',
    VpcId=vpc.id
)
vpc_security_group_id = response['GroupId']

In [28]:
# Create a database cluster, specifying an engine-mode of serverless:
db_cluster_identifier = 'database-cluster-01'

response = client.create_db_cluster(
    DBClusterIdentifier=db_cluster_identifier,
    Engine='aurora-postgresql',
    EngineMode='serverless',
    EngineVersion='10.14',
    MasterUsername='dbadmin',
    MasterUserPassword=admin_password,
    DBSubnetGroupName=subnet_group_name,
    VpcSecurityGroupIds=[
        vpc_security_group_id
    ]
)

In [30]:
# Wait for the database cluster to be available
print('Waiting for database cluster to be available...')
while True:
    response = client.describe_db_clusters(
        DBClusterIdentifier='database-cluster-01'
    )
    status = response['DBClusters'][0]['Status']
    if status == 'available':
        print('Database cluster is available.')
        break

Waiting for database cluster to be available...
Database cluster is available.


In [33]:
# Modify the database to automatically scale with new autoscaling capacity targets (8 min, 16 max) 
# and enable AutoPause after five minutes of inactivity

response = client.modify_db_cluster(
    DBClusterIdentifier=db_cluster_identifier,
    ScalingConfiguration={
        'MinCapacity': 8,
        'MaxCapacity': 16,
        'AutoPause': True,
        'SecondsUntilAutoPause': 300,
        'TimeoutAction': 'ForceApplyCapacityChange'
    }
)

In [35]:
# Describe the database cluster to get the endpoint
response = client.describe_db_clusters(
    DBClusterIdentifier=db_cluster_identifier
)

In [40]:
# Wait at least five minutes and observe that the database’s capacity has scaled down to 0:
# Capacity units are the number of Aurora Replicas that are available for use
print("Capacity units:", response['DBClusters'][0]['Capacity'])

Capacity units: 0


#### Create EC2 instance

In [59]:
print("En la terminal ejecute el siguiente comando:")
print("    python create_ec2_ssm.py --vpc {} --tag SSM-Instance-Region1".format(vpc.id))

En la terminal ejecute el siguiente comando:
    python create_ec2_ssm.py --vpc vpc-09c2992bbf8edd40f --tag SSM-Instance-Region1


In [62]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
instance_id = 'i-0d834f699966cd2f3' # Instance ID from the previous step
instance = ec2.Instance(instance_id)

In [65]:
# Grant your EC2 instance’s security group access to the default PostgreSQL port
response = ec2_client.authorize_security_group_ingress(
    GroupId=instance.security_groups[0]['GroupId'],
    IpProtocol='tcp',
    FromPort=5432,
    ToPort=5432
)

In [67]:
print("En la terminal ejecute el siguiente comando:")
print("    aws ssm start-session --target {}".format(instance_id))

En la terminal ejecute el siguiente comando:
    aws ssm start-session --target i-0d834f699966cd2f3


In [54]:
# List the endpoints for the database cluster
response = client.describe_db_clusters(
    DBClusterIdentifier=db_cluster_identifier
)
endpoints = response['DBClusters'][0]['Endpoint']

In [57]:
!pip install psycopg2 -q

In [78]:
# Connect to the database cluster using the endpoint
import psycopg2
from psycopg2 import Error

try:
    conn = psycopg2.connect(
        user="dbadmin",
        password=admin_password,
        host=endpoints,
        port="5432",
        database="postgres"
    )
    cursor = conn.cursor()
    print("PostgreSQL server information")
    print(conn.get_dsn_parameters(), "\n")
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

Error while connecting to PostgreSQL connection to server at "database-cluster-01.cluster-cfmpgxqena19.us-east-1.rds.amazonaws.com" (174.17.5.15), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "database-cluster-01.cluster-cfmpgxqena19.us-east-1.rds.amazonaws.com" (174.17.6.87), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?

